## This notebook shows how to create word vectors using word2vec pipeline

The notebook shows how to generate and save word embeddings from tweets. The tweets are preprocessed to handle URLs, Email ids ets., after which they are input into the word2vec vectorizer.

After features are generated, they are saved. They are re-loaded and used for several lookups, such as checking embeddings for words and sentences, examining words which are close to a given word in feature space.

### Set directories, import libraries

In [9]:
## Set training file name
working_dir = "D:\Sentiment140_Classification"

training_filename = os.path.join(working_dir, "training_text.csv")
embeddings_file_path = os.path.join(working_dir, "w2vec.txt")

In [2]:
# Show azureml-tatk version
!pip show azureml-tatk

Name: azureml-tatk
Version: 0.0.687318
Summary: Microsoft Azure Machine Learning Text Analytics Toolkit
Home-page: https://msdata.visualstudio.com/DefaultCollection/AlgorithmsAndDataScience/_git/TATK
Author: Microsoft Corporation
Author-email: azml-tatk@microsoft.com
License: UNKNOWN
Location: c:\users\remoteuser\appdata\local\amlworkbench\python\lib\site-packages
Requires: nose, azure-ml-api-sdk, bqplot, requests, numpy, scikit-learn, lxml, sklearn-crfsuite, pandas, docker, pyspark, pytest, gensim, validators, tensorflow-gpu, azure-storage, keras, scipy, ipython, unidecode, h5py, nltk, matplotlib, pdfminer.six


In [1]:
from __future__ import absolute_import
from __future__ import division
import collections
import math
import os
import sys
import random
import numpy as np
from six.moves import urllib
from six.moves import xrange  
import tensorflow as tf
from timeit import default_timer as timer
import pandas as pd
import re
import io
from nltk.tokenize import TweetTokenizer
import num2words

from tatk.feature_extraction.word2vec_vectorizer import Word2VecVectorizer

C:\Users\remoteuser\AppData\local\AmlWorkbench\Python\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Create functions for loading and preprocessing tweets

In [5]:
# Data processing and batch preparation
# In the following code, we replace Emails, URLS, emoticons etc with special labels
pos_emoticons=["(^.^)","(^-^)","(^_^)","(^_~)","(^3^)","(^o^)","(~_^)","*)",":)",":*",":-*",":]",":^)",":}",
               ":>",":3",":b",":-b",":c)",":D",":-D",":O",":-O",":o)",":p",":-p",":P",":-P",":Þ",":-Þ",":X",
               ":-X",";)",";-)",";]",";D","^)","^.~","_)m"," ~.^","<=8","<3","<333","=)","=///=","=]","=^_^=",
               "=<_<=","=>.<="," =>.>="," =3","=D","=p","0-0","0w0","8D","8O","B)","C:","d'-'","d(>w<)b",":-)",
               "d^_^b","qB-)","X3","xD","XD","XP","ʘ‿ʘ","❤","💜","💚","💕","💙","💛","💓","💝","💖","💞",
               "💘","💗","😗","😘","😙","😚","😻","😀","😁","😃","☺","😄","😆","😇","😉","😊","😋","😍",
               "😎","😏","😛","😜","😝","😮","😸","😹","😺","😻","😼","👍"]

neg_emoticons=["--!--","(,_,)","(-.-)","(._.)","(;.;)9","(>.<)","(>_<)","(>_>)","(¬_¬)","(X_X)",":&",":(",":'(",
               ":-(",":-/",":-@[1]",":[",":\\",":{",":<",":-9",":c",":S",";(",";*(",";_;","^>_>^","^o)","_|_",
               "`_´","</3","<=3","=/","=\\",">:(",">:-(","💔","☹️","😌","😒","😓","😔","😕","😖","😞","😟",
               "😠","😡","😢","😣","😤","😥","😦","😧","😨","😩","😪","😫","😬","😭","😯","😰","😱","😲",
               "😳","😴","😷","😾","😿","🙀","💀","👎"]

# Emails
emailsRegex=re.compile(r'[\w\.-]+@[\w\.-]+')

# Mentions
userMentionsRegex=re.compile(r'(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9]+)')

#Urls
urlsRegex=re.compile('r(f|ht)(tp)(s?)(://)(.*)[.|/][^ ]+') # It may not be handling all the cases like t.co without http

#Numerics
numsRegex=re.compile(r"\b\d+\b")

punctuationNotEmoticonsRegex=re.compile(r'(?<=\w)[^\s\w](?![^\s\w])')

emoticonsDict = {}
for i,each in enumerate(pos_emoticons):
    emoticonsDict[each]=' POS_EMOTICON_'+num2words.num2words(i).upper()+' '
    
for i,each in enumerate(neg_emoticons):
    emoticonsDict[each]=' NEG_EMOTICON_'+num2words.num2words(i).upper()+' '
    
# use these three lines to do the replacement
rep = dict((re.escape(k), v) for k, v in emoticonsDict.items())
emoticonsPattern = re.compile("|".join(rep.keys()))

# Function for reading in tweet training file
def read_tweets(filename):
    """Read the raw tweet data from a file. Replace Emails etc with special tokens """
    with open(filename, 'r') as f:
        all_lines=f.readlines()
        padded_lines=[]
        for line in all_lines:
            line = emoticonsPattern.sub(lambda m: rep[re.escape(m.group(0))], line.lower().strip())
            line = userMentionsRegex.sub(' USER ', line )
            line = emailsRegex.sub(' EMAIL ', line )
            line=urlsRegex.sub(' URL ', line)
            line=numsRegex.sub(' NUM ',line)
            line=punctuationNotEmoticonsRegex.sub(' PUN ',line)
            line=re.sub(r'(.)\1{2,}', r'\1\1',line)
            words_tokens=[token for token in TweetTokenizer().tokenize(line)]                    
            line= ' '.join(token for token in words_tokens )         
            padded_lines.append(line)
    return padded_lines

### Read in and preprocess tweets

In [7]:
tweets = read_tweets(training_filename)
df = pd.DataFrame({'tweets':tweets})
display(df[:5])

,tweets
0,damn fixtated on USER lovely thighs PUN hips o...
1,god bless firefox PUN s ' restore previous ses...
2,USER http://twitpic PUN com PUN 6vn4a - dang g...
3,USER hey PUN sorry u had to come back to work ...
4,bye mommy PUN we PUN ll miss you PUN


### Define & train word2vec vectorizer to generate word embeddings

In [11]:
word2vec = Word2VecVectorizer(input_col = 'tweets', output_col = '', embedding_size = 50, return_type = 'word_vector', context_window_size=5, min_df=5, num_workers=4)
word2vec_model = word2vec.tatk_fit(df)

Word2VecVectorizer::tatk_fit ==> start
vocabulary size =49517
Word2VecVectorizer::tatk_fit ==> end 	 Time taken: 1.08 mins


### Save word2vec embeddings in a txt file

In [12]:
word2vec_model.save_embeddings(embeddings_file_path)

Word2VecVectorizer::save_embeddings ==> start
Time taken: 0.03 mins
Word2VecVectorizer::save_embeddings ==> end


### Load the embeddings to memory with include_unk set to True to add OOV treatment

In [13]:
vectorizer = Word2VecVectorizer.load_embeddings(embeddings_file_path, include_unk = True, unk_method = 'rnd', unk_vector = None, unk_word = '<UNK>')

Word2VecVectorizer::load_embeddings ==> start
Time taken: 0.06 mins
Word2VecVectorizer::load_embeddings ==> end


### Embedding Lookup: Get word and subword indices.

In [15]:
df_predict = pd.DataFrame({'text' : ["I have fever", "this is a good book"]})
vectorizer.input_col = 'text'
vectorizer.output_col = 'indices'
vectorizer.return_type = 'word_index'
result = vectorizer.tatk_transform(df_predict)
display(result)

Word2VecVectorizer::tatk_transform ==> start
Word2VecVectorizer::tatk_transform ==> end 	 Time taken: 0.0 mins


,text,indices
0,I have fever,"[1, 24, 1187]"
1,this is a good book,"[36, 14, 6, 39, 485]"


### Embedding Lookup: Get word embeddings.

In [16]:
vectorizer.output_col = 'word_vector'
vectorizer.return_type = 'word_vector'
result = vectorizer.tatk_transform(df_predict)
display(result)

Word2VecVectorizer::tatk_transform ==> start
Word2VecVectorizer::tatk_transform ==> end 	 Time taken: 0.0 mins


,text,indices,word_vector
0,I have fever,"[1, 24, 1187]","[[-4.8849592208862305, -0.6744599938392639, -3..."
1,this is a good book,"[36, 14, 6, 39, 485]","[[1.7684129476547241, -3.304960012435913, -2.9..."


### Embedding Lookup: Get sentence embedding from word embeddings
This uses averaging of word vectors

In [17]:
vectorizer.output_col = 'sentence_vector'
vectorizer.return_type = 'sentence_vector'
result = vectorizer.tatk_transform(df_predict)
display(result)

Word2VecVectorizer::tatk_transform ==> start
Word2VecVectorizer::tatk_transform ==> end 	 Time taken: 0.0 mins


,text,indices,word_vector,sentence_vector
0,I have fever,"[1, 24, 1187]","[[-4.8849592208862305, -0.6744599938392639, -3...","[-2.7049067616462708, -1.0639479756355286, -2...."
1,this is a good book,"[36, 14, 6, 39, 485]","[[1.7684129476547241, -3.304960012435913, -2.9...","[0.5443949937820435, -1.2150852117687463, -1.0..."


### Embedding Lookup: Get most similar word to a given word

In [18]:
vectorizer.embedding_table.most_similar('fever')

[('migraine', 0.8524950742721558),
 ('headache', 0.8398715257644653),
 ('infection', 0.8327495455741882),
 ('migrane', 0.8285326361656189),
 ('toothache', 0.814940333366394),
 ('cough', 0.8102630376815796),
 ('asthma', 0.8058581948280334),
 ('bronchitis', 0.7783978581428528),
 ('sniffles', 0.7689926624298096),
 ('sinus', 0.7599035501480103)]